In [ ]:
%AddJar http://central.maven.org/maven2/com/google/code/gson/gson/2.2.4/gson-2.2.4.jar -f
%AddJar https://repo.eclipse.org/content/repositories/paho-releases/org/eclipse/paho/org.eclipse.paho.client.mqttv3/1.1.1/org.eclipse.paho.client.mqttv3-1.1.1.jar -f
%AddJar https://github.com/sathipal/spark-streaming-mqtt-with-security_2.10-1.3.0/releases/download/0.0.1/spark-streaming-mqtt-security_2.10-1.3.0-0.0.1.jar -f 
%AddJar http://central.maven.org/maven2/org/scalanlp/breeze_2.10/0.12/breeze_2.10-0.12.jar -f
%AddJar http://central.maven.org/maven2/com/datastax/cassandra/cassandra-driver-core/3.0.0-rc1/cassandra-driver-core-3.0.0-rc1.jar -f
%AddJar http://dl.bintray.com/spark-packages/maven/datastax/spark-cassandra-connector/1.6.0-s_2.10/spark-cassandra-connector-1.6.0-s_2.10.jar -f

In [ ]:

import scala.math.BigDecimal
import scala.io.Source
import org.eclipse.paho.client.mqttv3._
import org.eclipse.paho.client.mqttv3.persist.MemoryPersistence
import com.google.gson.JsonObject
import com.google.gson.JsonParser

import org.apache.spark.storage.StorageLevel
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.streaming.mqtt._
import org.apache.spark.SparkConf

import java.util.Map.Entry
import com.google.gson.JsonElement
import java.util.Set

import scala.collection.mutable.Map
import scala.collection.mutable.ArrayBuffer

import org.apache.spark.sql._

import com.datastax.spark.connector._

import breeze.linalg._


import com.datastax.spark.connector._
import com.datastax.spark.connector.cql._

import org.apache.spark.SparkContext
import scala.math._

In [ ]:
class JsonPackage() extends Serializable {

    
    var  _event : String  = ""
    var  _timestamp : String  = ""
    var  _severity : Int  = 0
    var  _temp : Int  = 0
    var  _precip : String = ""
    var  _construction : Int = 0
    
         
    def timestamp_= (value:String):Unit = _timestamp = value  
    def timestamp = _timestamp
  
    def temp_= (value:Int):Unit = _temp = value  
    def temp = _temp
    
    def precip_= (value:String):Unit = _precip = value  
    def precip = _precip
    
    def tweet = (value:String):Unit = _tweet = value  
    def tweet = _tweet
    
    override def toString: String =
    {
        
        var contents = "<Package>\n\t<Event>"+_event+"<\\Event>\n\t<Temp>"+_temp+"<\\Temp>\n\t<Precip>"+_precip+"<\\Precip>\n\t<Tweet>"+_tweet+"<\\Tweet>" 
        var closingtags = "\n<\\Package>"
        var value = contents+closingtags
        
        value
    }
    

}

In [ ]:
@transient val ssc = new StreamingContext(sc, Seconds(5))
ssc.checkpoint(".")

@transient val lines = MQTTUtils.createStream(ssc,
"ssl://hbp995.messaging.internetofthings.ibmcloud.com:8883", // Watson IoT Platform URL
                "iot-2/type/+/id/+/evt/+/fmt/+",                                // MQTT topic to receive the events
                "a:hbp995:random",                                          // Unique ID of the application
                "a-hbp995-t4eavmw5ru",                                                    // API-Key
                "IhYVjdS9&e0UQ7g_nC")  

@transient  val deviceMappedLines = lines.map(x => ((x.split(" ", 2)(0)).split("/")(4), x.split(" ", 2)(1)))
deviceMappedLines.print()

@transient val batch = deviceMappedLines.map(x => {
    var json = new JsonPackage()
	val payload = new JsonParser().parse(x._2).getAsJsonObject()
    val deviceObject = payload.get("d").getAsJsonObject()
	val setObj = deviceObject.entrySet()
    val itr = setObj.iterator()
    while(itr.hasNext()) {
 		val entry = itr.next()
        var index = entry.toString.split("=")(0)
        if(index == "type")
        {
           json.event = entry.getValue.getAsString() 
        }
        if(index == "tweet")
        {
           json.tweet = entry.getValue.getAsString() 
        }
        if(index == "temp")
        {
           json.temp = entry.getValue.getAsInt() 
        }
        if(index == "precip")
        {
           json.precip = entry.getValue.getAsString() 
        }

      
    }

    json
})

batch.foreachRDD(rdd => rdd.collect().foreach{json =>
                                              
   println(json.toString())
  })

ssc.start()
ssc.awaitTermination()
println("Finished")